# EnKF Experiments - GCS

## Imports

In [1]:
# Imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from scipy.stats import shapiro, kruskal, mode
import seaborn as sns
import sys

In [2]:
sys.path.append('../../../../stationsim/')
from ensemble_kalman_filter import (
    EnsembleKalmanFilter,
    AgentIncluder,
    Inflation,
    ExitRandomisation,
    GateEstimator
)
from stationsim_gcs_model import Model

In [3]:
sys.path.append('../')
from experiment_utils import Modeller, Processor, Visualiser

In [4]:
%matplotlib inline
np.random.seed(42)

## Data paths

In [5]:
data_dir = '../results/data/exp1/'
model_dir = '../results/models/exp1/'
fig_dir = '../results/figures/exp1/'

## Functions

In [6]:
def tidy_dataframe(df, independent_col: str, dependent_cols: list):
    output = list()
    
    for i, row in df.iterrows():
        for col in dependent_cols:
            d = {independent_col: row[independent_col],
                 'variable':  col,
                 'value': row[col]}
            output.append(d)
    
    output = pd.DataFrame(output)
    return output

In [7]:
def plot_path(output, aid, show_obs=True, show_od=True, show_fig=True,
              save_fig=False, ge=None, er=None):
    if save_fig and (ge is None or er is None):
        raise ValueError('Provide values for ge and er when saving figures')
    print(f'Agent {aid}')
    plt.figure(figsize=(10, 10))
    plt.plot(output[f'truth_x_{aid}'], output[f'truth_y_{aid}'], label=f'truth_{aid}', color='green')
    plt.plot(output[f'prior_x_{aid}'], output[f'prior_y_{aid}'], label=f'prior_{aid}', color='blue')
    plt.plot(output[f'posterior_x_{aid}'], output[f'posterior_y_{aid}'], label=f'posterior_{aid}', color='orange')

    if show_obs:
        plt.scatter(output[f'obs_x_{aid}'], output[f'obs_y_{aid}'], label=f'obs_{aid}', s=1, color='green')
        
    if show_od:
        plt.scatter(output[f'origin_x_{aid}'], output[f'origin_y_{aid}'], label=f'origin_{aid}', s=25, color='blue')
        plt.scatter(output[f'dest_x_{aid}'], output[f'dest_y_{aid}'], label=f'destination_{aid}', s=25, color='orange')

    plt.xlabel('$x$-location')
    plt.ylabel('$y$-location')
    plt.xlim((0, 740))
    plt.ylim((0, 700))
    plt.legend()
    
    if save_fig:
        plt.savefig(f'{fig_dir}path_agent_{aid}_{ge}_{er}.pdf')
    
    if show_fig:
        plt.show()
    else:
        plt.close()

In [8]:
def present_gate_estimates(enkf, agent_number, show_initial=True, show_true=True,
                           show_fig=True, save_fig=False, ge=None, er=None):
    if save_fig and (ge is None or er is None):
        raise ValueError('Provide values for ge and er when saving figures')
    
    idx = (2 * enkf.population_size) + agent_number
    true_gate = enkf.base_model.agents[agent_number].gate_out
    print(f'Base model gate value: {true_gate}')

    gate_vals = enkf.state_ensemble[idx]

    if hasattr(enkf, 'gate_angles'):
        gate_edges = enkf.gate_angles[true_gate]
        plt.xlabel('Target angle')
        print(f'Gate edge angles: {gate_edges}')
        if show_true:
            # Only need one label for legend
            plt.axvline(gate_edges[0], linestyle='dashed', c='black')
            plt.axvline(gate_edges[1], label='truth', linestyle='dashed', c='black')
    else:
        plt.xlabel('Target gate number')
        print(f'Ensemble modal gate value: {mode(gate_vals)}')
        if show_true:
            plt.axvline(true_gate, label='truth', linestyle='dashed', c='black')

    print(f'Gate value across ensemble: {gate_vals}')
    
    plt.hist(gate_vals, label='final', alpha=0.5, color='orange')
    if show_initial:
        plt.hist(enkf.initial_gates[agent_number], label='initial', alpha=0.5, color='blue')
    
    
    plt.ylabel('Frequency')
    plt.legend()

    if save_fig:
        plt.savefig(f'{fig_dir}gates_agent_{agent_number}_{ge}_{er}.pdf')

    if show_fig:
        plt.show()
    else:
        plt.close()

In [9]:
def master_func(p, e, a, s, ge, er, run_models=False, show_figs=False):
    if run_models:
        # Run modeller
        Modeller.run_experiment_1(pop_size=p, ensemble_size=e, assimilation_period=a, obs_noise_std=s,
                                  standardise_state=True, inclusion=AgentIncluder.MODE_EN,
                                  gate_estimator=ge, exit_randomisation=er)
    
    # Process results
    Processor.process_experiment_1(pop_size=p)
    
    # Read results
    results = pd.read_csv(data_dir + f'p{p}/metrics.csv')
    
    # Plot exit gate accuracy
    plt.figure()
    plt.plot(results['time'], results['exit_accuracy'])
    plt.xlabel('Time')
    plt.ylabel('Proportion of agents with correct gates')
    plt.savefig(f'{fig_dir}accuracy_{ge.name}_{er.name}.pdf')
    if show_figs:
        plt.show()
    else:
        plt.close()
    
    # Plot error
    plt.figure()
    plt.plot(results['time'], results['analysis'], label='posterior')
    plt.plot(results['time'], results['forecast'], label='prior')
    plt.legend()
    plt.xlabel('Time')
    plt.ylabel('Error in ensemble mean position')
    plt.savefig(f'{fig_dir}error_{ge.name}_{er.name}.pdf')
    if show_figs:
        plt.show()
    else:
        plt.close()
    
    # Read in enkf
    with open(f'../results/models/exp1/p{p}/model.pkl', 'rb') as f:
        enkf = pickle.load(f)
    
    # Plot gain matrix
    plt.figure()
    sns.heatmap(enkf.gain_matrix)
    plt.savefig(f'{fig_dir}gain_matrix_{ge.name}_{er.name}.pdf')
    if show_figs:
        plt.show()
    else:
        plt.close()
    
    # Process results
    output = list()
    for r in enkf.results:
        d = dict()
        # get time
        d['time'] = r['time']

        for i in range(enkf.population_size):        
            # get truth
            truth = r['ground_truth']
            d[f'truth_x_{i}'] = truth[2*i]
            d[f'truth_y_{i}'] = truth[(2*i)+1]

            # get prior
            prior = r['prior']
            d[f'prior_x_{i}'] = prior[i]
            d[f'prior_y_{i}'] = prior[i + enkf.population_size]

            # get posterior
            posterior = r['posterior']
            d[f'posterior_x_{i}'] = posterior[i]
            d[f'posterior_y_{i}'] = posterior[i + enkf.population_size]

            # get obs
            obs = r['observation']
            d[f'obs_x_{i}'] = obs[2*i]
            d[f'obs_y_{i}'] = obs[(2*i)+1]

            # get origin and destination
            origin = r['origin']
            d[f'origin_x_{i}'] = origin[2*i]
            d[f'origin_y_{i}'] = origin[(2*i)+1]
            destination = r['destination']
            d[f'dest_x_{i}'] = destination[2*i]
            d[f'dest_y_{i}'] = destination[(2*i)+1]

        output.append(d)

    output = pd.DataFrame(output)

    for i in range(enkf.population_size):
        plot_path(output, i, save_fig=True, ge=ge.name, er=er.name, show_fig=show_figs)
        present_gate_estimates(enkf, i, save_fig=True, ge=ge.name, er=er.name, show_fig=show_figs)
        print('\n')

## Experiment 1 - Setting Up EnKF

### Base model inclusion in error calculations

In [10]:
# # Set filter params
# p = 5
# e = 100
# a = 100
# s = 5
# ge = GateEstimator.ROUNDING
# er = ExitRandomisation.ALL_RANDOM

In [11]:
param_list = [{'pop_size': 5, 'ens_size': 100, 'assimilation_period': 100, 'noise_std': 5,
               'gate_estimator': GateEstimator.ROUNDING, 'exit_randomisation': ExitRandomisation.BY_AGENT},
              {'pop_size': 5, 'ens_size': 100, 'assimilation_period': 100, 'noise_std': 5,
               'gate_estimator': GateEstimator.ROUNDING, 'exit_randomisation': ExitRandomisation.ALL_RANDOM},
              {'pop_size': 5, 'ens_size': 100, 'assimilation_period': 100, 'noise_std': 5,
               'gate_estimator': GateEstimator.ROUNDING, 'exit_randomisation': ExitRandomisation.ADJACENT},
              {'pop_size': 5, 'ens_size': 100, 'assimilation_period': 100, 'noise_std': 5,
               'gate_estimator': GateEstimator.ANGLE, 'exit_randomisation': ExitRandomisation.BY_AGENT},
              {'pop_size': 5, 'ens_size': 100, 'assimilation_period': 100, 'noise_std': 5,
               'gate_estimator': GateEstimator.ANGLE, 'exit_randomisation': ExitRandomisation.ALL_RANDOM},
              {'pop_size': 5, 'ens_size': 100, 'assimilation_period': 100, 'noise_std': 5,
               'gate_estimator': GateEstimator.ANGLE, 'exit_randomisation': ExitRandomisation.ADJACENT}]              

In [12]:
for pl in param_list:
    master_func(p=pl['pop_size'], e=pl['ens_size'], a=pl['assimilation_period'],
                s=pl['noise_std'], ge=pl['gate_estimator'], er=pl['exit_randomisation'],
                run_models=True, show_figs=False)

/home/keiran/Documents/dev/dust/Projects/ABM_DA/experiments/enkf_experiments/results_3/noteooks/../../../../stationsim/ensemble_kalman_filter.py:215: RuntimeWarning: EnKF received unexpected attribute (vanilla_ensemble_size).
  warns.warn(w, RuntimeWarning)
/home/keiran/Documents/dev/dust/Projects/ABM_DA/experiments/enkf_experiments/results_3/noteooks/../../../../stationsim/ensemble_kalman_filter.py:215: RuntimeWarning: EnKF received unexpected attribute (inflation_rate).
  warns.warn(w, RuntimeWarning)


Running Ensemble Kalman Filter...
max_iterations:	10000
ensemble_size:	100
assimilation_period:	100
pop_size:	5
filter_type:	EnsembleKalmanFilterType.DUAL_EXIT
inclusion_type:	AgentIncluder.MODE_EN
ensemble_errors:	False


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [05:38<00:00, 29.53it/s]


Agent 0
Base model gate value: 0
Ensemble modal gate value: ModeResult(mode=array([3.]), count=array([100]))
Gate value across ensemble: [3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3.]


Agent 1
Base model gate value: 10
Ensemble modal gate value: ModeResult(mode=array([3.]), count=array([100]))
Gate value across ensemble: [3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3.]


Agent 2
Base model gate value: 8
Ensemble modal gate value: ModeResult(mode=array([1.]), count=array([100]))


/home/keiran/Documents/dev/dust/Projects/ABM_DA/experiments/enkf_experiments/results_3/noteooks/../../../../stationsim/ensemble_kalman_filter.py:215: RuntimeWarning: EnKF received unexpected attribute (vanilla_ensemble_size).
  warns.warn(w, RuntimeWarning)
/home/keiran/Documents/dev/dust/Projects/ABM_DA/experiments/enkf_experiments/results_3/noteooks/../../../../stationsim/ensemble_kalman_filter.py:215: RuntimeWarning: EnKF received unexpected attribute (inflation_rate).
  warns.warn(w, RuntimeWarning)


...

Agent 0
Base model gate value: 2
Gate edge angles: (0.8148014516014535, 1.3325515686161515)
Gate value across ensemble: [ 0.86711665  0.49555167 -0.55638113  1.29354607 -0.62830324 -1.09626643
  0.28336777  0.20969778  0.81523642  0.13432144 -0.74450675  0.49555167
  1.01562518 -0.12916323  0.25743372  0.30440836 -0.16768564 -0.19385699
 -1.95189006 -0.66426696 -2.18929303  0.30884006  0.64160073 -0.89871213
  1.07909218 -0.60425825 -0.4081491   0.13432144  0.39111268  1.0988644
  1.90109368 -0.02702045  1.29287513 -0.35756302 -0.14897684  0.46523196
  0.81480145 -0.59609446  0.13432144 -1.15544781  0.61253756 -0.03894722
 -0.99424945  0.25346232 -0.4081491   0.75484388  0.75246354  0.13432144
  0.33518642 -0.17267846 -0.8047629  -1.35672325  0.67635061 -1.81835225
 -0.10077559 -0.94714122  0.17891855 -0.02702045  0.1659789   0.1559775
 -0.54636002  0.30767955  0.61253756  0.75762753 -0.91939979 -0.54636002
 -0.25609247 -0.96374098 -0.94976152 -0.39337996 -0.63372425  0.49555167
  0.33

/home/keiran/Documents/dev/dust/Projects/ABM_DA/experiments/enkf_experiments/results_3/noteooks/../../../../stationsim/ensemble_kalman_filter.py:215: RuntimeWarning: EnKF received unexpected attribute (vanilla_ensemble_size).
  warns.warn(w, RuntimeWarning)
/home/keiran/Documents/dev/dust/Projects/ABM_DA/experiments/enkf_experiments/results_3/noteooks/../../../../stationsim/ensemble_kalman_filter.py:215: RuntimeWarning: EnKF received unexpected attribute (inflation_rate).
  warns.warn(w, RuntimeWarning)


Base model gate value: 0
Gate edge angles: (3.0072712116368248, -2.646040980103965)
Gate value across ensemble: [-0.80016609 -0.6661299  -0.75330701 -0.96696366 -0.96144282 -0.97024067
 -0.78423128 -0.90343329  0.29543584 -0.75931228  0.40022809 -0.75762753
 -0.78452467 -0.847759   -0.92076704 -0.2198172  -0.89928483 -0.27800875
 -0.75698119 -0.81971095 -0.78254853 -0.86734525 -0.03711391  0.22147202
 -0.84615233 -1.03981605 -0.93888771 -1.83611946 -0.74784963 -0.8901249
 -2.64604098 -1.04706861 -0.85362945 -0.68426115 -0.70883279 -0.78707192
 -0.76155415 -1.30241589 -0.95006172 -0.77966756 -0.99914965 -0.02702045
 -0.75762753 -0.35090211  0.49555167 -0.58817443 -1.01387202 -1.64747413
 -0.94714189 -1.0081862  -0.89035093 -0.76153638 -1.00186176  0.74053217
 -0.84527626 -1.28304247 -1.05788813 -1.03066287 -1.62343937 -1.06576068
 -0.75659003 -0.7798539  -0.69564856 -1.04116535 -0.75762753 -0.94625253
 -1.85085039 -0.74642297 -0.91895527 -1.0387454  -0.82499172 -0.79028557
 -0.75133985 

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [03:14<00:00, 51.38it/s]


Agent 0
Base model gate value: 8
Gate edge angles: (-1.0845536937073916, -1.5707963267948966)
Gate value across ensemble: [-2.66118257  2.00664333 -1.37147582 -2.93585698  0.14505122 -1.14427763
 -1.6617099  -2.89712282 -0.39276955 -1.95726842 -1.43748268  3.01731156
  0.63644088 -1.66522277 -1.40944778 -1.3263256   1.00175006 -2.65277428
 -1.76256159 -1.86047573 -0.92817035  1.90109368  2.21263491 -3.02407641
  0.13432144 -3.05459284  0.49274743 -2.30140875  3.01044965 -0.96790527
  2.38396513 -1.67815162 -1.26705259 -0.4081491   0.61253756  0.68746167
  2.10672588  0.49673603 -2.86138076 -3.04002377 -1.09773872  1.3167693
  3.00727121  3.09300606 -0.3625282   1.94699621 -2.67135663 -2.9603642
 -1.50563311  1.90109368 -2.25458212  3.0110725  -2.95344425  3.00727121
  0.19190888 -3.01778683 -0.30027625  0.89316881 -0.59472285 -2.79941068
  0.43709516 -1.35816337 -2.89686877 -1.85939747 -2.18081803  1.90109368
  1.90109368 -0.40700061 -1.30531479 -2.97762055  3.03096369 -1.81906148
  2.

In [13]:
# param_list = [{'pop_size': 5, 'ens_size': 100, 'assimilation_period': 100, 'noise_std': 5,
#                'gate_estimator': GateEstimator.ANGLE, 'exit_randomisation': ExitRandomisation.ADJACENT}]              

In [14]:
# for pl in param_list:
#     master_func(p=pl['pop_size'], e=pl['ens_size'], a=pl['assimilation_period'],
#                 s=pl['noise_std'], ge=pl['gate_estimator'], er=pl['exit_randomisation'],
#                 run_models=True, show_figs=False)

In [15]:
# Modeller.run_experiment_1(pop_size=p, ensemble_size=e, assimilation_period=a, obs_noise_std=s,
#                           standardise_state=True, inclusion=AgentIncluder.MODE_EN,
#                           gate_estimator=ge, exit_randomisation=er)

In [16]:
# Processor.process_experiment_1(pop_size=p)

#### Read data

In [17]:
# results = pd.read_csv(data_dir + f'p{p}/metrics.csv')
# results.head()

In [18]:
# plt.figure()
# plt.plot(results['time'], results['exit_accuracy'])
# plt.xlabel('Time')
# plt.ylabel('Proportion of agents with correct gates')
# plt.savefig(f'{fig_dir}accuracy_{ge.name}_{er.name}.pdf')
# plt.show()

In [19]:
# plt.figure()
# plt.plot(results['time'], results['analysis'], label='posterior')
# plt.plot(results['time'], results['forecast'], label='prior')
# plt.legend()
# plt.xlabel('Time')
# plt.ylabel('Error in ensemble mean position')
# plt.savefig(f'{fig_dir}error_{ge.name}_{er.name}.pdf')
# plt.show()

In [20]:
# with open(f'../results/models/exp1/p{p}/model.pkl', 'rb') as f:
#     enkf = pickle.load(f)

In [21]:
# plt.figure()
# sns.heatmap(enkf.gain_matrix)
# plt.savefig(f'{fig_dir}gain_matrix_{ge.name}_{er.name}.pdf')
# plt.show()

In [22]:
# print(enkf.gain_matrix)

In [23]:
# output = list()
# for r in enkf.results:
#     d = dict()
#     # get time
#     d['time'] = r['time']
    
#     for i in range(enkf.population_size):        
#         # get truth
#         truth = r['ground_truth']
#         d[f'truth_x_{i}'] = truth[2*i]
#         d[f'truth_y_{i}'] = truth[(2*i)+1]

#         # get prior
#         prior = r['prior']
#         d[f'prior_x_{i}'] = prior[i]
#         d[f'prior_y_{i}'] = prior[i + enkf.population_size]

#         # get posterior
#         posterior = r['posterior']
#         d[f'posterior_x_{i}'] = posterior[i]
#         d[f'posterior_y_{i}'] = posterior[i + enkf.population_size]

#         # get obs
#         obs = r['observation']
#         d[f'obs_x_{i}'] = obs[2*i]
#         d[f'obs_y_{i}'] = obs[(2*i)+1]

#         # get origin and destination
#         origin = r['origin']
#         d[f'origin_x_{i}'] = origin[2*i]
#         d[f'origin_y_{i}'] = origin[(2*i)+1]
#         destination = r['destination']
#         d[f'dest_x_{i}'] = destination[2*i]
#         d[f'dest_y_{i}'] = destination[(2*i)+1]
    
#     output.append(d)
    
# output = pd.DataFrame(output)


In [24]:
# output.head()

In [25]:
# for i in range(enkf.population_size):
#     plot_path(output, i, save_fig=True, ge=ge.name, er=er.name)
#     present_gate_estimates(i, save_fig=True, ge=ge.name, er=er.name)
#     print('\n')

* Look at standardisation
* scaling variables

In [26]:
# t = -1

# plt.figure(figsize=(10, 10))

# truth = enkf.results[t]['ground_truth']
# truth_x = truth[::2]
# truth_y = truth[1::2]
# plt.scatter(truth_x, truth_y, color='green', s=25)



# for i in range(e):
#     prior_label = f'prior_{i}'
#     posterior_label = f'posterior_{i}'
    
#     prior = enkf.results[t][prior_label]
#     posterior = enkf.results[t][posterior_label]
    
    
#     prior_x = prior[:2]
#     prior_y = prior[2:4]
#     posterior_x = posterior[:2]
#     posterior_y = posterior[2:4]
    
#     plt.scatter(prior_x, prior_y, color='blue', s=1)
#     plt.scatter(posterior_x, posterior_y, color='red', s=1)

# plt.xlim((0, 740))
# plt.ylim((0, 700))
# plt.show()